# Lecture 11: Adding more integration

## Recap

Last time we discussed the trapezoidal rule for numerical integration. 

We found that 

$$
\large 
\int_{a}^{b} f(x) \, \mathrm{d} x \ = \ \frac{1}{n}\ \left( \ \frac{f(x_{0}) + f(x_{n})}{2} \ + \  \sum_{k=1}^{n-1}\ f(x_{k}) \ \right) \ + \  \mathcal{O}(n^{-2}) \quad \mathrm{as} \quad  n \ \to \ \infty 
$$

<br>

The term $\mathcal{O}(n^{-2})$ means

<br>

$$
\large
\lim_{n\to \infty} \ n^{2}\, \left[ \ \int_{a}^{b} f(x) \, \mathrm{d} x - \frac{1}{n}\ \left( \ \frac{f(x_{0}) + f(x_{n})}{2} \ + \  \sum_{k=1}^{n-1}\ f(x_{k}) \ \right) \right] \ = \ \textit{Constant}  
$$

<br>

## Big *O*

In fact, anything 

$$
\large X(n) \ \ = \  \ \mathcal{O}(\, Y(n)\, )  \quad \mathrm{as} \quad  n \ \to \ \infty 
$$

means 

$$
\large
\lim_{n\to \infty} \frac{|X(n)|}{|Y(n)|} \ = \ \textit{Constant} \ \le \ \infty.
$$


<br>

This is also the same as 

$$
\large X(n) \ \ \sim \  \  Y(n)   \quad \mathrm{as} \quad  n \ \to \ \infty. 
$$

<br>



Today, we're going to look at another way of approximating a function for integration.

## Simpson's rule: piecewise quadratic functions

Rather than fitting $f(x)$ to a straight line $L(x)$, we can fit it to a quadratic function,

$$\large 
Q(x) = \alpha\,  x^2 + \beta\,  x + \gamma.$$

The fitting rules are 

$$
\large 
f(a) = Q(a),  \quad \quad f(b) = Q(b), \quad \quad f( m ) = Q(m) $$

where 

$$
\large 
m \ = \ \frac{a+b}{2}
$$

<br>

We can solve for $\alpha, \beta, \gamma$ in $Q(x)$. The algebra is a little messy, but it cleans up a bit

<br>

$$
\large 
Q(x) \ = \ \frac{2(x-b)(x-m)}{(b-a)^{2}}\ f(a) - 4\ \frac{(x-b)(x-a)}{{(b-a)^{2}}} \ f(m) + \frac{2(x-a) (x-m)}{{(b-a)^{2}}}  \ f(b)
$$

<br>

We can integrate the quadratic to get the approximate integral

<br>

$$
\large 
\int_{a}^{b} Q(x) \ \mathrm{d} x \ = \ \frac{b-a}{6}\left(\  f(a)  \ +  \ 4\, f(m)  \ +  \ f(b)\  \right)
$$

## (Composite) Simpson's rule

We can string together a bunch of Simpson's intervals and get a composite rule:

<br>

$$
\large
\int_{a}^{b} f(x) \, \mathrm{d} x \ = \ \frac{h}{3}\ \left( \ f(x_{0}) + f(x_{n}) \ + \  2\sum_{k=1}^{n/2-1}\ f(x_{2k}) + 4 \sum_{k=1}^{n/2}\ f(x_{2k-1}) \ \right) \ + \  \mathcal{O}(n^{-4}) \quad \mathrm{as} \quad  n \ \to \ \infty 
$$

<br>

Where

$$
\large
h \ = \ \frac{b-a}{n}
$$

is the spacing between successive points.

Note that single Simpson's rule requires **three** function evaluations and **two** intervals ($[a,m],[m,b]$) -- instead of two function evaluations for one interval $[a,b]$ for the trapezoidal rule. 

Therefore, 

<h3><center> For Simpson's Rule, the number of points must be odd; the number of intervals must be even.</center></h3>

Be careful of the indexing choice! The number $n$ here is the number of *intervals* -- one less than the number of points.

## Simpson's vs. Trapezoid (blame Eric)

<img src="https://frinkiac.com/meme/S03E12/872714.jpg?b64lines=VGhpcyBpc24ndCBzb21lIHNoYWR5IAppbnRlZ3JhdGlvbiBzY2hlbWUgeW91J3ZlIApiZWVuIGhlYXJpbmcgYWJvdXQ=" width="400" height="400" />

<img src="https://frinkiac.com/meme/S03E12/875417.jpg?b64lines=Tm8gc2lyLiBPdXIgbW9kZWwKaXMgdGhlIHRyYXBlem9pZCwKIHdpdGggZ3VhcmFudGVlZCAKMm5kIG9yZGVyIHJldHVybg==" width="400" height="400" />

([The skit](https://www.youtube.com/watch?v=uuVh36vQkSI))

Let's get comparing!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# This grabs a Simpson's rule integrator from Scipy
from scipy.integrate import simps as simp

In [ ]:
def trap(y,x): 
    """Integrate y values against x values using trapezoidal rule."""
    return (x[1]-x[0])*( 0.5*(y[0] + y[-1]) + np.sum(y[1:-1]) )
            
def g(n,parity): 
    if parity ==  'odd': parity = 1
    if parity == 'even': parity = 0
    return np.linspace(0,1,2*(n//2)+parity)

def h(n): return 1/(2*(n//2))

def Q(f,x):
    """A quadratic interpolation polynomial."""
    a, m, b = x[0], x[len(x)//2], x[-1]
    
    qa = 2*(m-x)*(b-x)/(b-a)**2
    qm = 4*(x-a)*(b-x)/(b-a)**2
    qb = 2*(x-a)*(x-m)/(b-a)**2
    
    return qa*f(a) + qm*f(m) + qb*f(b)
    

### Absolute error

If we want to compare the size of two quantities $A \approx B$, we can subtract them and take the absolute value.

$$
\large | \, A \, - \, B \, | \quad \quad \text{(absolute error)}
$$

This measure of error only make sense if $A$ and $B$ are close to 1.  But it doesn't work very well if $A$ and $B$ are small or large. For example, if

$$
\large
A \ = \ 10^{-4} \quad \text{and} \quad B \ = \ 10^{-6} \quad \implies \quad \large | \, A \, - \, B \, | \ \approx \ |\, A \, | \ = \ 10^{-4}
$$

These number are both small. The only thing that the absolute error tells us is that one of the numbers is small. Not how ***close*** they are to each other. In this example, these numbers are not close whatsoever. 


### Relative error

I think you all probably know the solution in your head. The subtle aspect of the relative error is that we should think of one of the quantities as "more correct" than the other. The relative error is when we have some kind of ***benchmark***, $B$ and we have another quantity, $A$ that we want to compare, 

<br>

$$
\large \frac{| \, A \, - \, B \, |}{|\,B\,|} \quad \quad \text{(relative error)}
$$

<br>

In the above example, 

$$
\large
A \ = \ 10^{-4} \quad \text{and} \quad B \ = \ 10^{-6} \quad \implies \quad \large \frac{| \, A \, - \, B \, |}{|\,B\,|} \ \approx \ \frac{|\, A \, |}{|\,B\,| } \ = \ 100
$$


<br> 

The relative error is what we should use when computing the accuracy of integrals.

These are a few helper functions to make plots and such. 

In [ ]:
from helper import *

In [ ]:
def errors(f,I,nmin=10,nmax=1000,even=False,style='log-log',powers={-2:'green',-4:'orange'}):
    """Calculate and plot errors of trapezoidal and Simpson's rule."""
    
    n = np.arange(nmin+(nmin%2)+1,nmax,2) # odd numbers only
    
    error_Trap, error_Simp = np.zeros(len(n)), np.zeros(len(n))
    
    if even: error_Simp_even = np.zeros(len(n))
    
    for k in range(len(n)):
        
        x = g(n[k],'odd')
        
        T = trap(f(x),x)
        S = simp(f(x),x)

        error_Trap[k] = err(T,I)
        error_Simp[k] = err(S,I)
        
        if even:
            x = g(n[k],'even')
            S_even = simp(f(x),x)
            error_Simp_even[k] = err(S_even,I)

        
    show = show_plots(style=style,xy=['n','e(n)'],title='$|\,approx.\, -\, exact\,|\,/\,|\,exact\,|$')
    
    show(n,error_Trap,color='blue')
    show(n,error_Simp,color='red')
    
    if style == 'log-log': 
        for p in powers: show(n,1.5*n**float(p),color=powers[p])
    
    if even:
        show(n,error_Simp_even,color='black')
        if style == 'log-log': show(n,2.0*n**(-3.),color='grey')
    
    return show
    

Like last time, start with 

$$
\large
f(x) = \frac{x}{\sqrt{2}} \  + \ \frac{1}{5} \,\sin( \, 9\, x \, ). $$

With 

$$
\large
\int_{0}^{1} \, f(x) \, \mathrm{d} x \ = \ \frac{1}{180} \left(4+45 \sqrt{2}-4 \cos (9)\right) \ \approx \ 0.396023
$$

In [ ]:
def f(x): return np.sqrt(0.5)*x + 0.2*np.sin(9*x)

I = (4 + 45*np.sqrt(2) - 4*np.cos(9))/180

def q(x): return Q(f,x)

Let's plot Simpson's approximation, and highlight the different intervals:

In [ ]:
n = 10

show = show_plots(linewidth=3.5)
x = np.linspace(0,1,101)
show(x,f(x))

for k in range(n):
    interval = [k/n, (k+1)/n]
    x = np.linspace(*interval,101)
    show(x,q(x))
# plot the interval points    
points = np.linspace(0,1,2*n+1)
plt.plot(points,f(points),'k.',markersize=7);

## Compare Trapezoidal, and Simpson's

Let's look at the errors of the trapezoidal and Simpson's rule.

This will loop over different values of $n$ and make a log-log plot of the errors for both methods.

It's important to mention that I used an ***odd*** number of grid points, and an ***even*** number of intervals. Otherwise, Simpson's rule doesn't work very well.

In [ ]:
show = errors(f,I);
show['Trap.$\,\sim n^{-2}$','Simp.$\,\sim n^{-4}$']

### Even-number Simpson's?

Last time, I messed up the trapezoidal rule by not taking care of the endpoints properly. This made $\mathcal{O}(n^{-2})$ error become $\mathcal{O}(n^{-1})$. 

What if we use an ***even*** number of grid points, and an ***odd*** number of intervals?

In [ ]:
show = errors(f,I,even=True);
show['Trap.$\,\sim n^{-2}$','Simp(even).$\,\sim n^{-3}$','Simp(odd).$\,\sim n^{-4}$']

So the trapezoidal rule is second order, and Simpson's rule is 4th order!

Is this always the case?

### Periodic function

Last time, we saw something funny when we tried periodic functions.
Let's use

$$
\large
f(x) \ = \ \frac{1}{2} \ + \  \frac{3}{10 \, + \, 8 \cos (\, 2 \, \pi \, x ) }
$$

since

$$
\large
\int_{0}^{1} \, f(x) \, \mathrm{d} x \ = \ 1
$$


In [ ]:
def f(x): return 0.5 + 3/(8*np.cos(2*np.pi*x) + 10)

def q(x): return Q(f,x)

In [ ]:
n = 15

x = np.linspace(0,1,101)
show = show_plots()
show(x,f(x),color='blue')

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    show(x,q(x),color='red')
    
points = np.linspace(0,1,2*n+1)
plt.plot(points,f(points),'k.',markersize=10)    

### Compare Simpson's and Trapezoidal rule for a periodic function

In [ ]:
show = errors(f,1,style='log',nmax=50);
show['Trap. $\,\sim \, 2^{-n}$','Simp. $\,\sim \, \sqrt{2}^{-n}$']

n = np.arange(10,50,2)
show(n,2.0**(-n))
show(n,2.0**(-n//2))

# Summary so far:

## For most functions:

###  $\quad $    Trapezoidal rule: error $\sim n^{-2}$

###  $\quad $    Simpson's rule: error $\sim n^{-4}$


## For *periodic* functions:

###  $\quad $    Trapezoidal rule: error $\sim \tau^{-n}$

###  $\quad $    Simpson's rule: error $\sim \sigma^{-n}$

###  $\quad $    We don't usually know $\tau$, $\sigma$  ahead of time. But we can expect $\sigma \ \approx \sqrt{\tau}$.


## List slices

In the composite Simpson's rule, we want to treat the endpoints, even indices, and odd indices differently.
How can we extract these things efficiently?
By **slicing**!

When you see something like `[a:b:c]` at the end of a list (or array, or anything that can be iterated, really) it's called a `slice`, ([see here](https://stackoverflow.com/questions/509211/understanding-pythons-slice-notation)) and takes the form:

    [start:stop:step]

It takes all the elements from index `start`, 
up to and not including index `stop`,
in steps of `step`. 
Some important points:

- A negative `stop` counts backwards from the end position. 
- A negative `step` takes steps backward.
- Omitting `start` will begin from 0.
- Omitting `end` will continue until the list ends.
- Omitting `step` will take a step of 1.

Test it out on the list `[0,1,2,3,4,5]` if you're not sure.

In [9]:
x = list(range(6))
print('x:')
print(x)
print('')

print('The following should give the same thing for s = 1,2,3:')
print('')
print('x[0:len(x):s]: ')
for s in range(1,4): print(x[0:len(x):s])
    
print('')
print('x[0::s]: ')
for s in range(1,4): print(x[0::s])

print('')
print('The following should give the same thing for s = 1,2,3:')
print('')
print('x[0:len(x)-1:s]: ')
for s in range(1,4): print(x[0:len(x)-1:s])

print('')
print('x[0:-1:s]:')
for s in range(1,4): print(x[0:-1:s])

x:
[0, 1, 2, 3, 4, 5]

The following should give the same thing for s = 1,2,3:

x[0:len(x):s]: 
[0, 1, 2, 3, 4, 5]
[0, 2, 4]
[0, 3]

x[0::s]: 
[0, 1, 2, 3, 4, 5]
[0, 2, 4]
[0, 3]

The following should give the same thing for s = 1,2,3:

x[0:len(x)-1:s]: 
[0, 1, 2, 3, 4]
[0, 2, 4]
[0, 3]

x[0:-1:s]:
[0, 1, 2, 3, 4]
[0, 2, 4]
[0, 3]


## DYI Simspon's rule (with slicing)

In [10]:
def simp(y, x):
    n = len(x) - 1
    h = (x[-1] - x[0])/n
    odds  = y[1::2]
    evens = y[2:-1:2]
    return (h/3)*(y[0]+y[-1] + 4*np.sum(odds) + 2*np.sum(evens))

## Approximating polynomials

An obvious, and important, `class` of functions to consider is the class of powers of $x$:

$$
\large
f(x) \ = \ x^{p}, \quad \text{where} \quad  p > -1.
$$

where 


$$
\large
\int_{0}^{1} f(x) \, \mathrm{d} x \ =  \  \frac{1}{p+1}
$$

Taking the hint from above,
and recalling our object oriented programming from previous lectures,
using a `class` is a good way to define functions with a parameter. 
The `class` consists of all functions of a given form. 
An `object` is a particular one of those functions. 

In [ ]:
class power():
    
    def __init__(self,p):
        self.p = p

    def __call__(self,x):
        return x**self.p
    
    def __str__(self):
        if self.p == 1: return 'x'
        if type(self.p)==int: return 'x^{{{:d}}}'.format(self.p)
        else: return 'x^{{{:.2f}}}'.format(self.p)
    
    def area(self,a=0,b=1):
        q = self.p + 1
        return (b**q - a**q)/q

    def show(self,a=0,b=1,n=2000):
        x = np.linspace(a,b,n)
        y = self(x)
        S = show_plots(title='$f(x) \, = \, x^{%s}$'% self.p,titlesize=20,xy=['x','y'])
        S(x,y,color='blue')
        return S
    
    def q(self, x):
        return Q(self, x)

There are some convenient methods to make nice plots.

Let's use them!

### Example 1: Line

Well if we can't do this properly, we're in trouble

In [ ]:
n = 2

f = power(1)
show = f.show()
for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

Because we're returning the initial plot from `self.show`, 
we can add things to it later:

In [ ]:
show = errors(f,f.area());
show['Trap.$\,\sim ??$','Simp.$\,\sim ??$']

OK, well that was a bit silly - of course a line can fit a line, and a parabola can too (it's degenerate now).
But we know that our functions are behaving correctly.

What about a parabola?

### Example 2: A parabola

This defines $f(x) = x^{2}$. 

In [ ]:
n = 3

f = power(2)

show = f.show()

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

In [ ]:
show = errors(f,f.area());
show['Trap.$\,\sim n^{-2}$','Simp.$\,\sim ??$']

Well, that was boring.
No surprise that a quadratic function can exactly approximate a parabola.

The trapezoidal rule is back to our guaranteed second order convergence.

### Example 3: A cubic

Well, I guess we know what's going to happen for $x^3$.

In [ ]:
n = 1

f = power(3)

show = f.show()

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

In [ ]:
show = errors(f,f.area());
show['Trap.$\,\sim n^{-2}$','Simp.$\,\sim ??$']

Eh?

Even though a parabola doesn't exactly approximate a cubic - Simpson's method gives the exact answer for the integral!

**Why?!**

### Example 4: A quartic

Well, what about $x^4$?

In [ ]:
n = 1

f = power(4)

show = f.show()

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

In [ ]:
show = errors(f,f.area());
show['Trap.$\,\sim n^{-2}$','Simp.$\,\sim n^{-4}$']

OK, that's a bit more expected. Our expected second order trapezoidal rule, and fourth order Simpson's rule.

But is the salesman genuine?
Are we *guaranteed* at least second and fourth order convergence?

<img src="https://frinkiac.com/meme/S03E12/876668.jpg?b64lines=" width="500" height="400" />


### Example 1/2: A square root

Let's look at $f(x) = \sqrt{x}$. 
It's a simple enough function to want to approximate.

In [ ]:
n = 4

f = power(1/2)

show = f.show()

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

Hmm, doesn't look great around $x = 0$...

In [ ]:
show = errors(f,f.area(),powers={-1.5:'green'});
show['Trap.$\,\sim n^{-3/2}$','Simp.$\,\sim n^{-3/2}$']

<img src="https://frinkiac.com/meme/S03E12/873332.jpg?b64lines=" width="500" height="400" />

<img src="https://frinkiac.com/img/S03E12/883559.jpg" width="500" height="400" />

He lied!

Well, what convergence *are* we getting?

Why are we only getting $n^{-1.5}$ convergence!

### Example 1/10: A tenth root?

In [ ]:
n = 4

f = power(1/10)

show = f.show()

for k in range(n):
    x = np.linspace(k/n,(k+1)/n,101)
    plt.plot(x,f.q(x),color='green',linewidth=3)

points = np.linspace(0,1,2*n+1)

plt.plot(points,f(points),linewidth=3,color='red')    
plt.plot(points,f(points),'k.')

In [ ]:
show = errors(f,f.area(),powers={-11/10:'green'});
show['Trap.$\,\sim n^{-11/10}$','Simp.$\,\sim n^{-11/10}$']

Well there's definitely a pattern going on here.

Now, this is a mathematical computing course - so we need to be understand how to examine these things theoretically too.

### For $ p < 1 $, both the Trapezoidal and Simpson's rule give the same error scaling:

$$
\Large \text{error} \ \sim \ n^{- (1+p)}
$$
 
Therefore 

$$\large p=\frac{1}{10}   \quad \implies \quad n^{-\tfrac{3}{2}}, \quad \quad \quad \quad \quad p=\frac{1}{2}  \quad \implies \quad n^{-\tfrac{11}{10}}$$. 
